Q2 image classification using resnet50

Importing the libraries

In [ ]:
import matplotlib.pyplot as plt #to plot validation loss curves
import numpy as np
import pandas as pd
import tensorflow as tf  #i have used tf as the main ml framework
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Dense, Flatten

from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


Loading mnist data into pandas dataframe


In [ ]:
train_df = pd.read_csv('fashion-mnist_train.csv')
test_df = pd.read_csv('fashion-mnist_test.csv')


preparing data : 1. Normalising the pixel value between 0,1 from 0,255 - for easier computation

2. Converting the category labels to encodings using to_categorical function as to categorical_crossentropy function to work

In [ ]:
X = train_df.drop('label', axis=1).values / 255.0  # normalizes to [0, 1] as pixel values is bw 0 , 255
y = to_categorical(train_df['label'].values, num_classes=10) #

Splitting the training dataset into training and validation set 90-10

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)


Preprocessing the image data : Reshaping flat 784 array to 28x28x1 (1 for as grayscale)

Converting grayscale to rgb using duplication (grayscale_to_rgb function) for resnet

Resizing to 224x224

In [ ]:
def preprocess(image, label):
    image = tf.reshape(image, [28, 28, 1])
    image = tf.image.grayscale_to_rgb(image)
    image = tf.image.resize(image, [224, 224])
    return image, label

Making a tf.data pipeline as its easy to use and allows parellel processing in batches

-from_tensor_slices converts numpy arrays to tf.data objects
- map function applies the preprocess function on these objects
- that AUTOTUNE stuff is basically for tf to decide how do to the parellel processing (how many thread to use and stuff)
-shuffle is for randomising in each epoch
- prefetch is for preloading the next batch while the current is on training (for gpu utilisation)


In [ ]:
BATCH_SIZE =  100

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.shuffle(1024).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_dataset = val_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

Creating a Sequential model using keras , will add layers in this

In [ ]:
resnet_model = Sequential()


Loading the Resnet50 Model from keras without the top layer.

this model has weights pretrained from imagenet dataset

defining the input size and the no. of classes

In [ ]:
pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(224,224,3),
                   pooling='avg',classes=10,
                   weights='imagenet')

Freezing the backbone, setting model layers as untrainable

In [ ]:
for layer in pretrained_model.layers:
        layer.trainable=False

Adding the layers resnet model --> Linear block ----> FC layer  ---> Output layer (softmax)

In [ ]:
resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(10, activation='softmax'))

resnet_model.summary()

compiling the model with Adam as optimizer and loss function method as categorical_crossentropy as we have many classes


Fitting the model for validation set with 10 epochs.

In [ ]:
resnet_model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

epochs=10

history = resnet_model.fit(train_dataset, validation_data=val_dataset, epochs=epochs)

plotting the Training/Validation Accuracy and Loss curves

In [ ]:
fig1 = plt.gcf()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.axis(ymin=0.4,ymax=1)
plt.grid()
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.grid()
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

***Fine-Tuning***

Unfreezing the last 10 layers of the model for training and again compiler and fitting the model on validation set

Also i set the learning rate lower this time  = 10^-5

In [ ]:
for layer in pretrained_model.layers[-10:]:
    layer.trainable = True

resnet_model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])


history = resnet_model.fit(train_dataset, validation_data=val_dataset, epochs=10)

plotting the Training/Validation Accuracy and Loss curves after fine tuning

In [ ]:
fig1 = plt.gcf()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.axis(ymin=0.4,ymax=1)
plt.grid()
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.grid()
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()


Finally Predicting the Test dataset images

In [ ]:
loss , accuracy = resnet_model.evaluate(test_dataset)
print(f"Accuracy : {accuracy}")
